In [43]:
# %% [markdown]
# # Integración de Pinecone con modelo Logistic Regression para etiquetado multilabel

# %%
# Instalar librerías necesarias (una vez)
# %pip install pinecone
# %pip install sentence-transformers
# %pip install joblib

# %%
import os
import pandas as pd
import pinecone
import joblib
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from pinecone import Pinecone, ServerlessSpec
import configparser
import time
from pinecone import exceptions


DEFAULT_BOOK_TITLE = "Messi: Edición revisada y actualizada (Biografías y memorias)"
DEFAULT_BOOK_BLURB = (
    "Leo Messi es el jugador de fútbol más conocido del planeta, pero también un enigma como persona, por su hermetismo. Esta biografía, que fue publicada por primera vez en 2014, y posteriormente actualizada en 2018, se presenta de nuevo en una edición que recoge lo más relevante de los últimos años del jugador en el Fútbol Club Barcelona. \n\n"
    "En esta nueva edición, el autor repasa lo más destacado desde aquel fatídico Mundial de Brasil hasta el final de la temporada 2017/18, así como su paso por el Mundial de Rusia y por la Copa América 2021, que coincidía con el momento en que expiraba su contrato con el Fútbol Club Barcelona, y que convirtió al astro argentino en foco de todas las miradas, generando una enorme expectación.\n\n"
    "En agosto de 2021, se anunció el desenlace que parecía imposible: Messi no pudo renovar en el Barça y se anunció su fichaje por el PSG. ¿Qué pasó? ¿Cómo es posible que, queriendo quedarse, tuviera que salir?"
)

In [44]:
# Get the pinecone API key from config.cfg
config = configparser.ConfigParser()
config.read('../config.cfg')
PINECONE_API_KEY = config['pinecone']['api_key']
PINECONE_ENV = config['pinecone']['environment']

# %%
# 1. Inicializa Pinecone (usa tu API Key personal desde https://app.pinecone.io/)
try:
    pc = Pinecone(api_key=PINECONE_API_KEY)
    print(f"✅ Connected to Pinecone!")
except Exception as e:
    print("❌ Failed to connect:", e)

index_name = "book-embeddings"
#index_name = "quickstart"

from pinecone import exceptions

try:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )
    print("✅ Índice creado correctamente.")
except exceptions.PineconeApiException as e:
    if "ALREADY_EXISTS" in str(e):
        print("📌 El índice ya existe. Usando el existente.")
    else:
        raise e


index = pc.Index(index_name)

# Borrar vectores del índice si ya había
stats = index.describe_index_stats()
if stats.get('total_vector_count', 0) > 0:
    index.delete(delete_all=True)
    print("🗑️ Vectores antiguos borrados.")
else:
    print("ℹ️ Índice ya estaba vacío.")





✅ Connected to Pinecone!
📌 El índice ya existe. Usando el existente.
ℹ️ Índice ya estaba vacío.


In [45]:
# 3. Carga tus modelos ya entrenados
model = joblib.load("../model/book_tagging_pipeline_sentence_bert.joblib")
clf = joblib.load("../model/book_tagging_pipeline.joblib")
mlb = joblib.load("../model/book_tagging_pipeline_mlb.joblib")

# %%
# 4. Carga tus libros originales y sube a Pinecone
df = pd.read_csv("../data/processed/books.csv")
df['text'] = df['book_title'].fillna('') + '. ' + df['blurb'].fillna('')

# Ensure 'tags' column does not contain NaN values
df['tags'] = df['tags'].fillna('')

print(df['tags'].head(3))

0    abandonment-in-god, biography, catholic, chełm...
1    _pilar, _sindy, abandonment-in-god, anxiety, a...
2    catholic, chełmińska, christ, christianity, de...
Name: tags, dtype: object


In [46]:
# Generate embeddings (if not already saved)
X_embeddings = model.encode(df['text'].tolist(), show_progress_bar=True)

# Prepare data for Pinecone
pinecone_data = []
for idx, (vec, tags) in enumerate(zip(X_embeddings, df['tags'])):
    # Ensure tags are strings and handle NaN or invalid values
    if isinstance(tags, list):
        tags = ','.join(tags)  # Convert list to comma-separated string
    elif pd.isna(tags) or not isinstance(tags, str):
        tags = ''  # Replace NaN or invalid values with an empty string

    pinecone_data.append((
        str(idx),
        vec.tolist(),
        {"tags": tags}
    ))

# Delete existing data in Pinecone (if any)
try:
    index.delete(delete_all=True)
except pinecone.exceptions.NotFoundException:
    print("⚠️ Namespace not found. Skipping deletion.")

# Upload embeddings to Pinecone in batches of 1000
batch_size = 1000
for i in range(0, len(pinecone_data), batch_size):
    batch = pinecone_data[i:i + batch_size]
    index.upsert(vectors=batch)

print("✅ Embeddings uploaded to Pinecone.")

# Ver cuántos vectores hay almacenados
stats = index.describe_index_stats()
print(f"📊 Vectores cargados en el índice: {stats['total_vector_count']}")

# Comprueba que la consulta devuelve algo:
query_vector = X_embeddings[0]  # el primer libro
res = index.query(vector=query_vector.tolist(), top_k=3, include_metadata=True)
print(f"res = {res}")

Batches:   0%|          | 0/37 [00:00<?, ?it/s]

⚠️ Namespace not found. Skipping deletion.
✅ Embeddings uploaded to Pinecone.
📊 Vectores cargados en el índice: 0
res = {'matches': [], 'namespace': '', 'usage': {'read_units': 1}}


In [47]:
# 5. Función de predicción combinada (ensemble)
def predict_with_ensemble(title, blurb, top_k=5, threshold=0.3):
    text = title + ". " + blurb
    embedding = model.encode([text])[0]
    
    # A. Predict con Logistic Regression
    probs = np.array([estimator.predict_proba(embedding.reshape(1, -1))[0][1]
                      for estimator in clf.estimators_])
    pred_lr = (probs >= threshold).astype(int)  # Ensure this remains a NumPy array
    pred_lr = np.array([pred_lr])  # Ensure pred_lr is 2D for mlb.inverse_transform
    tags_lr = mlb.inverse_transform(pred_lr)[0]
    
    # B. Predict con Pinecone (top-K más cercanos)
    pinecone_result = index.query(vector=embedding.tolist(), top_k=top_k, include_metadata=True)
    tag_counter = {}
    for match in pinecone_result.matches:
        if 'tags' in match.metadata and match.metadata['tags']:
            for tag in match.metadata['tags'].split(','):
                tag_counter[tag.strip()] = tag_counter.get(tag.strip(), 0) + 1
    
    tags_pinecone = [tag for tag, count in tag_counter.items() if count >= 1]

    # C. Fusionar (por unión o intersección, aquí hacemos unión)
    # final_tags = sorted(set(tags_lr) | set(tags_pinecone))
    # por intersección: 
    final_tags = sorted(set(tags_lr) & set(tags_pinecone))

    return {
        "tags_logistic": sorted(tags_lr),
        "tags_pinecone": sorted(tags_pinecone),
        "tags_fusion": final_tags
    }

# %%
# 6. Prueba con libro desconocido para ti
result = predict_with_ensemble(DEFAULT_BOOK_TITLE, DEFAULT_BOOK_BLURB)

print("Tags por Logistic Regression:", result["tags_logistic"])
print("Tags por Pinecone:", result["tags_pinecone"])
print("Tags combinados (fusión):", result["tags_fusion"])

Tags por Logistic Regression: ['biography']
Tags por Pinecone: []
Tags combinados (fusión): []
